In [1]:
# import libraries

import pickle
import numpy as np
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import pandas as pd

In [2]:
# 讀取資料

with open('x_train_new.pickle', 'rb') as handle:
    x_train = pickle.load(handle)
with open('y_train_new.pickle', 'rb') as handle:
    y_train = pickle.load(handle)
with open('x_test_new.pickle', 'rb') as handle:
    x_test = pickle.load(handle)
with open('y_test_new.pickle', 'rb') as handle:
    y_test = pickle.load(handle)            

In [3]:
x_test = torch.cat((x_train, x_test), 0)
y_test = np.concatenate((y_train, y_test))

In [4]:
# load model
xgb_model_loaded = pickle.load(open("xgb_model.pkl", "rb"))

# Make predictions on the validation set
y_test_pred = xgb_model_loaded.predict(x_test)

# Calculate evaluation metrics
acc = accuracy_score(y_test, y_test_pred)                             # Calculate the accuracy score
f1 = f1_score(y_test, y_test_pred, average='weighted')                # Calculate the weighted F1-score
precision = precision_score(y_test, y_test_pred, average='weighted')  # Calculate the weighted precision
recall = recall_score(y_test, y_test_pred, average='weighted')        # Calculate the weighted recall
cm = confusion_matrix(y_test, y_test_pred)                            # Calculate the confusion matrix

# Print the evaluation metrics
print(f"Accuracy: {acc:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print("Confusion Matrix:")
print(cm)

Accuracy: 0.50
F1-score: 0.50
Precision: 0.52
Recall: 0.50
Confusion Matrix:
[[ 5  4  2  1]
 [ 7 22  4  4]
 [ 3  6 14  4]
 [ 2  2 11  9]]


In [5]:
with open("indices_train_new", "rb") as fp:
    indices_train_new = pickle.load(fp)
with open("indices_test_new", "rb") as fp:
    indices_test_new = pickle.load(fp)

In [6]:
indices_all = indices_train_new + indices_test_new

In [7]:
df = pd.DataFrame({'indices': indices_all, 'y_test': y_test+1, 'y_test_pred': y_test_pred+1})
df.sort_values(by=['indices'], inplace=True)
df['match'] = (df['y_test'] == df['y_test_pred']).map({True: 'T', False: 'F'})
df = df.iloc[:,1:].reset_index(drop=True)
df

,y_test,y_test_pred,match
0,2,2,T
1,1,2,F
2,3,3,T
3,2,2,T
4,2,2,T
...,...,...,...
95,4,3,F
96,2,2,T
97,3,3,T
98,1,2,F


In [8]:
df_new_sample = pd.read_excel("df_new_sample.xlsx")
df_new_company = pd.read_excel("df_new_company.xlsx")
sample_idx = df_new_sample.iloc[:,0]
df_sample_company = df_new_company.loc[sample_idx]
df_sample_company = df_sample_company.iloc[:,1:].reset_index(drop=True)
report = pd.concat([df_sample_company,df],axis=1)
report

,Company Name,Security Tickers,Exchange:Ticker,y_test,y_test_pred,match
0,"ONEOK, Inc. (NYSE:OKE)",NYSE:OKE; WBAG:OKE; XTRA:ONK; BOVESPA:O1KE34; ...,OKE,2,2,T
1,Honeywell International Inc. (NasdaqGS:HON),NasdaqGS:HON; SNSE:HON; SNSE:HONCL; WBAG:HON; ...,HON,1,2,F
2,Ciena Corporation (NYSE:CIEN),NYSE:CIEN; XTRA:CIE1; DB:CIE1; LSE:0HYA; BMV:C...,CIEN,3,3,T
3,Carlisle Companies Incorporated (NYSE:CSL),NYSE:CSL; DB:CLE,CSL,2,2,T
4,Carlisle Companies Incorporated (NYSE:CSL),NYSE:CSL; DB:CLE,CSL,2,2,T
...,...,...,...,...,...,...
95,Columbus McKinnon Corporation (NasdaqGS:CMCO),NasdaqGS:CMCO; DB:VC3,CMCO,4,3,F
96,Tractor Supply Company (NasdaqGS:TSCO),NasdaqGS:TSCO; WBAG:TSCO; XTRA:TR4; DB:TR4; BM...,TSCO,2,2,T
97,"AMN Healthcare Services, Inc. (NYSE:AMN)",NYSE:AMN; BMV:AMN *; DB:HCQ,AMN,3,3,T
98,General Dynamics Corporation (NYSE:GD),NYSE:GD; WBAG:GEDY; XTRA:GDX; BMV:GD *; BOVESP...,GD,1,2,F


In [9]:
report.to_excel("report.xlsx")